In [1]:
# basic DS imports
import pandas as pd
import numpy as np
from pydataset import data

# Spark for Python Imports
import pyspark
import pyspark.sql.functions as F

# Shhhhhhhhh....
import warnings
warnings.filterwarnings('ignore')

spark = pyspark.sql.SparkSession.builder.getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/10/21 09:15:11 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


#### Create a spark data frame that contains your favorite programming languages.

- The name of the column should be language
- View the schema of the dataframe
- Output the shape of the dataframe
- Show the first 5 records in the dataframe


In [2]:
lang = ['Python', 'SQL', 'PHP', 'CSS', 'JavaScript', 'R', 'Scala']

In [3]:
languages = pd.Series(lang)
df = pd.DataFrame(languages)
df.head()
df = df.rename(columns= {0: "language"})
df.head()

,language
0,Python
1,SQL
2,PHP
3,CSS
4,JavaScript


In [4]:
df = spark.createDataFrame(df)
type(df)

pyspark.sql.dataframe.DataFrame

In [5]:
df.printSchema()

root
 |-- language: string (nullable = true)



In [6]:
# Janky way to find the shape??
(df.count(), len(df.columns))

(7, 1)

In [7]:
df.show(5)

+----------+
|  language|
+----------+
|    Python|
|       SQL|
|       PHP|
|       CSS|
|JavaScript|
+----------+
only showing top 5 rows




#### Load the mpg dataset as a spark dataframe.

- Create 1 column of output that contains a message like the one below:


> The 1999 audi a4 has a 4 cylinder engine.
> For each vehicle.

- Transform the trans column so that it only contains either manual or auto.


In [8]:
mpg = spark.createDataFrame(data('mpg'))
mpg.show(5)

+------------+-----+-----+----+---+----------+---+---+---+---+-------+
|manufacturer|model|displ|year|cyl|     trans|drv|cty|hwy| fl|  class|
+------------+-----+-----+----+---+----------+---+---+---+---+-------+
|        audi|   a4|  1.8|1999|  4|  auto(l5)|  f| 18| 29|  p|compact|
|        audi|   a4|  1.8|1999|  4|manual(m5)|  f| 21| 29|  p|compact|
|        audi|   a4|  2.0|2008|  4|manual(m6)|  f| 20| 31|  p|compact|
|        audi|   a4|  2.0|2008|  4|  auto(av)|  f| 21| 30|  p|compact|
|        audi|   a4|  2.8|1999|  6|  auto(l5)|  f| 16| 26|  p|compact|
+------------+-----+-----+----+---+----------+---+---+---+---+-------+
only showing top 5 rows



In [9]:
mpg.select(F.concat(F.lit("The "), mpg.year,F.lit(" "), mpg.manufacturer, F.lit(" "), mpg.model, F.lit(" has a "), mpg.cyl, F.lit(" cylinder engine.")).alias('cyl_info')).sort(F.col('year'), F.col('manufacturer')).show(truncate=False)

+--------------------------------------------------------------+
|cyl_info                                                      |
+--------------------------------------------------------------+
|The 1999 audi a4 has a 6 cylinder engine.                     |
|The 1999 audi a4 quattro has a 6 cylinder engine.             |
|The 1999 audi a4 quattro has a 6 cylinder engine.             |
|The 1999 audi a4 has a 4 cylinder engine.                     |
|The 1999 audi a4 quattro has a 4 cylinder engine.             |
|The 1999 audi a4 has a 4 cylinder engine.                     |
|The 1999 audi a4 quattro has a 4 cylinder engine.             |
|The 1999 audi a4 has a 6 cylinder engine.                     |
|The 1999 audi a6 quattro has a 6 cylinder engine.             |
|The 1999 chevrolet corvette has a 8 cylinder engine.          |
|The 1999 chevrolet c1500 suburban 2wd has a 8 cylinder engine.|
|The 1999 chevrolet malibu has a 6 cylinder engine.            |
|The 1999 chevrolet malib

In [10]:
mpg.select(mpg.trans).show(5)

+----------+
|     trans|
+----------+
|  auto(l5)|
|manual(m5)|
|manual(m6)|
|  auto(av)|
|  auto(l5)|
+----------+
only showing top 5 rows



In [11]:
mpg.select('trans', F.regexp_extract('trans',r"^(\w+)\b\W", 1).alias('transmission')).show()

+----------+------------+
|     trans|transmission|
+----------+------------+
|  auto(l5)|        auto|
|manual(m5)|      manual|
|manual(m6)|      manual|
|  auto(av)|        auto|
|  auto(l5)|        auto|
|manual(m5)|      manual|
|  auto(av)|        auto|
|manual(m5)|      manual|
|  auto(l5)|        auto|
|manual(m6)|      manual|
|  auto(s6)|        auto|
|  auto(l5)|        auto|
|manual(m5)|      manual|
|  auto(s6)|        auto|
|manual(m6)|      manual|
|  auto(l5)|        auto|
|  auto(s6)|        auto|
|  auto(s6)|        auto|
|  auto(l4)|        auto|
|  auto(l4)|        auto|
+----------+------------+
only showing top 20 rows




#### Load the tips dataset as a spark dataframe.

- What percentage of observations are smokers?
- Create a column that contains the tip percentage
- Calculate the average tip percentage for each combination of sex and smoker.


In [12]:
tips = spark.createDataFrame(data('tips'))
tips.show(5)

+----------+----+------+------+---+------+----+
|total_bill| tip|   sex|smoker|day|  time|size|
+----------+----+------+------+---+------+----+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|
+----------+----+------+------+---+------+----+
only showing top 5 rows



In [13]:
print(f"percentage of smokers : {round(tips.where(tips['smoker']=='Yes').count()/tips.count(),2)}")

percentage of smokers : 0.38


In [14]:
tips = tips.withColumn('tip_percentage', F.expr('ROUND((tip/(total_bill - tip) * 100),1)'))
tips.show()

+----------+----+------+------+---+------+----+--------------+
|total_bill| tip|   sex|smoker|day|  time|size|tip_percentage|
+----------+----+------+------+---+------+----+--------------+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|           6.3|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|          19.1|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|          20.0|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|          16.2|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|          17.2|
|     25.29|4.71|  Male|    No|Sun|Dinner|   4|          22.9|
|      8.77| 2.0|  Male|    No|Sun|Dinner|   2|          29.5|
|     26.88|3.12|  Male|    No|Sun|Dinner|   4|          13.1|
|     15.04|1.96|  Male|    No|Sun|Dinner|   2|          15.0|
|     14.78|3.23|  Male|    No|Sun|Dinner|   2|          28.0|
|     10.27|1.71|  Male|    No|Sun|Dinner|   2|          20.0|
|     35.26| 5.0|Female|    No|Sun|Dinner|   4|          16.5|
|     15.42|1.57|  Male|    No|Sun|Dinner|   2|        

In [15]:
print(f"average tip % for female smokers : ")
tips.filter(tips.sex =='Female').where(tips['smoker']=='Yes').select(F.avg(tips.tip_percentage)).show()
print(f"average tip % for female non-smokers : ")
tips.filter(tips.sex =='Female').where(tips['smoker']=='No').select(F.avg(tips.tip_percentage)).show()
print(f"average tip % for male smokers : ")
tips.filter(tips.sex =='Male').where(tips['smoker']=='Yes').select(F.avg(tips.tip_percentage)).show()
print(f"average tip % for male non-smokers : ")
tips.filter(tips.sex =='Male').where(tips['smoker']=='No').select(F.avg(tips.tip_percentage)).show()

average tip % for female smokers : 
+-------------------+
|avg(tip_percentage)|
+-------------------+
| 23.309090909090912|
+-------------------+

average tip % for female non-smokers : 
+-------------------+
|avg(tip_percentage)|
+-------------------+
| 18.829629629629633|
+-------------------+

average tip % for male smokers : 
+-------------------+
|avg(tip_percentage)|
+-------------------+
|  20.99833333333333|
+-------------------+

average tip % for male non-smokers : 
+-------------------+
|avg(tip_percentage)|
+-------------------+
|  19.44020618556701|
+-------------------+




#### Use the seattle weather dataset referenced in the lesson to answer the questions below.

- Convert the temperatures to fahrenheit.
- Which month has the most rain, on average?
- Which year was the windiest?
- What is the most frequent type of weather in January?
- What is the average high and low temperature on sunny days in July in 2013 and 2014?
- What percentage of days were rainy in q3 of 2015?
- For each year, find what percentage of days it rained (had non-zero precipitation).

In [74]:
from vega_datasets import data

weather = data.seattle_weather().assign(date=lambda df: df.date.astype(str))
weather = spark.createDataFrame(weather)

In [75]:
weather.show(5)

+----------+-------------+--------+--------+----+-------+
|      date|precipitation|temp_max|temp_min|wind|weather|
+----------+-------------+--------+--------+----+-------+
|2012-01-01|          0.0|    12.8|     5.0| 4.7|drizzle|
|2012-01-02|         10.9|    10.6|     2.8| 4.5|   rain|
|2012-01-03|          0.8|    11.7|     7.2| 2.3|   rain|
|2012-01-04|         20.3|    12.2|     5.6| 4.7|   rain|
|2012-01-05|          1.3|     8.9|     2.8| 6.1|   rain|
+----------+-------------+--------+--------+----+-------+
only showing top 5 rows



In [76]:
weather = weather.withColumn('temp_max_f', F.expr('ROUND(temp_max * 1.8 + 32, 2)')).drop('temp_max')
weather = weather.withColumn('temp_min_f', F.expr('ROUND(temp_min * 1.8 + 32, 2)')).drop('temp_min')
weather.show()

+----------+-------------+----+-------+----------+----------+
|      date|precipitation|wind|weather|temp_max_f|temp_min_f|
+----------+-------------+----+-------+----------+----------+
|2012-01-01|          0.0| 4.7|drizzle|     55.04|      41.0|
|2012-01-02|         10.9| 4.5|   rain|     51.08|     37.04|
|2012-01-03|          0.8| 2.3|   rain|     53.06|     44.96|
|2012-01-04|         20.3| 4.7|   rain|     53.96|     42.08|
|2012-01-05|          1.3| 6.1|   rain|     48.02|     37.04|
|2012-01-06|          2.5| 2.2|   rain|     39.92|     35.96|
|2012-01-07|          0.0| 2.3|   rain|     44.96|     37.04|
|2012-01-08|          0.0| 2.0|    sun|      50.0|     37.04|
|2012-01-09|          4.3| 3.4|   rain|     48.92|      41.0|
|2012-01-10|          1.0| 3.4|   rain|     42.98|     33.08|
|2012-01-11|          0.0| 5.1|    sun|     42.98|     30.02|
|2012-01-12|          0.0| 1.9|    sun|     42.98|     28.94|
|2012-01-13|          0.0| 1.3|    sun|      41.0|     26.96|
|2012-01

In [96]:
weather.withColumn("month", F.month("date")).groupBy("month").agg(F.avg("precipitation").alias("average_rainfall")).agg(F.max('average_rainfall')).show()

+---------------------+
|max(average_rainfall)|
+---------------------+
|    5.354166666666667|
+---------------------+



- Which year was the windiest?


In [135]:
weather.withColumn('year',F.year(weather.date)).groupby( F.col('year')).agg(F.mean(weather.wind).alias('avg_wind')).sort(F.col('avg_wind').desc()).show(1)

+----+------------------+
|year|          avg_wind|
+----+------------------+
|2012|3.4008196721311483|
+----+------------------+
only showing top 1 row



In [137]:
## can also use .first but I don't like how it looks
weather.withColumn('year',F.year(weather.date)).groupby( F.col('year')).agg(F.mean(weather.wind).alias('avg_wind')).sort(F.col('avg_wind').desc()).first()

Row(year=2012, avg_wind=3.4008196721311483)

In [97]:
weather.withColumn("year", F.year("date")).groupBy("year").agg(F.avg("wind").alias("yearly_wind_avg")).agg(F.max('yearly_wind_avg')).show()

+--------------------+
|max(yearly_wind_avg)|
+--------------------+
|  3.4008196721311483|
+--------------------+



- What is the most frequent type of weather in January?


In [140]:
weather.filter(F.month('date') == 1).groupBy("weather").count().sort(F.col('count').desc()).show(1)

## PAY ATTENTION TO WHAT YOUR COLUMNS ARE CALLED, sorting by weather when you really want the "Count" column that shows up below, won't get you the results you want.

+-------+-----+
|weather|count|
+-------+-----+
|    fog|   38|
+-------+-----+
only showing top 1 row



- What is the average high and low temperature on sunny days in July in 2013 and 2014?


In [141]:
(
    weather.filter(F.month('date') == 7)
    .filter(F.year('date')>2012)
    .filter(F.year('date')<2015)
    .filter(F.col('weather') == F.lit('sun'))
    .agg(
        F.avg('temp_max_f').alias('average_high'),
        F.avg('temp_min_f').alias('average_low')
    )
    .show()
)

+-----------------+-----------------+
|     average_high|      average_low|
+-----------------+-----------------+
|80.29192307692308|57.52884615384615|
+-----------------+-----------------+



In [142]:
# july_weather.filter(F.year('date') == 2013).show(5)

- What percentage of days were rainy in q3 of 2015?


In [145]:
### Looking at documentation, you can see quarter function.

In [144]:
(
    weather.filter(F.year('date') == 2015)
    .filter(F.quarter('date')==3)
    .select(F.when(
        F.col('weather')=='rain',1).otherwise(0).alias('rain'))
    .agg(F.mean('rain'))
    .show()
)

+--------------------+
|           avg(rain)|
+--------------------+
|0.021739130434782608|
+--------------------+



- For each year, find what percentage of days it rained (had non-zero precipitation).

In [149]:
(
    weather.withColumn(
        'year', F.year('date')
    )
    .select(F.when(
        F.col('precipitation') > 0, 1
    ).otherwise(0).alias('did_rain'),'year')
    .groupby('year')
    .agg(F.mean('did_rain'))
    .show()
)

+----+-------------------+
|year|      avg(did_rain)|
+----+-------------------+
|2012|0.48360655737704916|
|2013|0.41643835616438357|
|2014|  0.410958904109589|
|2015|0.39452054794520547|
+----+-------------------+



In [150]:
# Spark's verbage and structure makes my head hurt....